In [ ]:
!unzip /content/NewR22.zip

Archive:  /content/NewR22.zip
   creating: NewR22/
   creating: NewR22/test/
   creating: NewR22/test/Defective/
  inflating: NewR22/test/Defective/118.jpg  
  inflating: NewR22/test/Defective/122.jpg  
  inflating: NewR22/test/Defective/251.jpg  
  inflating: NewR22/test/Defective/253.jpg  
  inflating: NewR22/test/Defective/256.jpg  
  inflating: NewR22/test/Defective/257.jpg  
  inflating: NewR22/test/Defective/258.jpg  
  inflating: NewR22/test/Defective/259.jpg  
  inflating: NewR22/test/Defective/260.jpg  
  inflating: NewR22/test/Defective/261.jpg  
  inflating: NewR22/test/Defective/262.jpg  
  inflating: NewR22/test/Defective/265.jpg  
  inflating: NewR22/test/Defective/266.jpg  
  inflating: NewR22/test/Defective/267.jpg  
  inflating: NewR22/test/Defective/268.jpg  
  inflating: NewR22/test/Defective/269.jpg  
  inflating: NewR22/test/Defective/270.jpg  
  inflating: NewR22/test/Defective/271.jpg  
  inflating: NewR22/test/Defective/272.jpg  
  inflating: NewR22/test/Defecti

In [ ]:

import os
from PIL import Image

def get_image_size(image_path):
    try:
        img = Image.open(image_path)
        width, height = img.size
        return width, height
    except FileNotFoundError:
        return None, None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None


width, height = get_image_size('/content/NewR22/test/Normal/28.jpg')

if width is not None and height is not None:
    print(f"Image size: {width} x {height}")
else:
    print(f"Could not determine image size or file not found.")


Image size: 224 x 224


###RailTrack_Detection_VGG16_98.2%

In [ ]:
# Standard libraries for file handling and numeric operations
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import numpy as np

# Enable GPU if available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

# Configuration parameters
NUM_CLASSES = 2                 # Binary classification (Normal vs Defective)
BATCH_SIZE = 32
EPOCHS = 20 #50
LR = 1e-4                       # Learning rate
WEIGHT_DECAY = 0.01            # L2 regularization
MODELS = ['vgg16', 'resnet50', 'inception_v3', 'efficientnet_b0']  # Model list

# Data preprocessing and augmentation
def get_transforms(model_name):
    input_size = 299 if model_name == 'inception_v3' else 224  # Inception requires 299x299
    return {
        'train': transforms.Compose([
            transforms.Resize((input_size, input_size)),              # Resize to model's expected size
            transforms.RandomResizedCrop(input_size, scale=(0.8, 1.0)),  # Random crop
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.2),
            transforms.RandomRotation(degrees=20),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
            transforms.ToTensor(),                                    # Convert to tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406],          # Normalize using ImageNet stats
                                 std=[0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
    }

# Create and fine-tune a pre-trained model
def create_model(name):
    if name == 'vgg16':
        model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        model.classifier[6] = nn.Linear(4096, NUM_CLASSES)
    elif name == 'resnet50':
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        model.fc = nn.Linear(2048, NUM_CLASSES)
    elif name == 'inception_v3':
        model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT, aux_logits=True)
        model.fc = nn.Linear(2048, NUM_CLASSES)
        model.AuxLogits.fc = nn.Linear(768, NUM_CLASSES)  # Auxiliary classifier used during training
    elif name == 'efficientnet_b0':
        model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        model.classifier[1] = nn.Linear(1280, NUM_CLASSES)
    else:
        raise ValueError("Unsupported model")
    return model.to(DEVICE)

# Training loop for a single model
def train_model(model_name):
    print(f"Training: {model_name.upper()}")
    transforms_dict = get_transforms(model_name)

    # Load dataset
    train_dataset = datasets.ImageFolder('/content/NewR22/train', transform=transforms_dict['train'])
    test_dataset = datasets.ImageFolder('/content/NewR22/test', transform=transforms_dict['test'])

    # Prepare dataloaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    # Initialize model
    model = create_model(model_name)
    is_inception = model_name == 'inception_v3'
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

    best_acc = 0.0  # Track best validation accuracy

    for epoch in range(EPOCHS):
        model.train()
        epoch_loss = 0.0
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS}')

        for inputs, labels in pbar:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            # MixUp augmentation
            alpha = 0.2
            lam = np.random.beta(alpha, alpha)
            index = torch.randperm(inputs.size(0)).to(DEVICE)
            mixed_inputs = lam * inputs + (1 - lam) * inputs[index]
            labels_a, labels_b = labels, labels[index]

            optimizer.zero_grad()

            if is_inception:
                # InceptionV3 returns a tuple (main_output, aux_output)
                outputs, aux_outputs = model(mixed_inputs)
                loss_main = lam * criterion(outputs, labels_a) + (1 - lam) * criterion(outputs, labels_b)
                loss_aux = lam * criterion(aux_outputs, labels_a) + (1 - lam) * criterion(aux_outputs, labels_b)
                loss = loss_main + 0.4 * loss_aux  # Combined loss
            else:
                outputs = model(mixed_inputs)
                loss = lam * criterion(outputs, labels_a) + (1 - lam) * criterion(outputs, labels_b)

            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Prevent exploding gradients
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_postfix(loss=loss.item())

        # Validation phase
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                if isinstance(outputs, tuple):  # For InceptionV3
                    outputs = outputs[0]
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        print(f"Validation Accuracy: {val_acc:.2f}%")

        # Save the best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), f'best_{model_name}.pth')

        scheduler.step()

    return best_acc

# Train and evaluate all models
results = {}
for model_name in MODELS:
    acc = train_model(model_name)
    results[model_name] = acc

# Print final results summary
print("\nFinal Results:")
for name, acc in results.items():
    print(f"{name.upper():<15}: {acc:.2f}%")

# Identify best model
best_model = max(results, key=results.get)
print(f"\nBest Model: {best_model.upper()} with {results[best_model]:.2f}% accuracy")


Using device: cuda
Training: VGG16


Epoch 1/20: 100%|██████████| 14/14 [00:07<00:00,  1.77it/s, loss=0.804]


Validation Accuracy: 81.98%


Epoch 2/20: 100%|██████████| 14/14 [00:07<00:00,  1.77it/s, loss=0.327]


Validation Accuracy: 95.50%


...

Validation Accuracy: 92.79%


Epoch 19/20: 100%|██████████| 14/14 [00:04<00:00,  3.18it/s, loss=0.456]


Validation Accuracy: 91.89%


Epoch 20/20: 100%|██████████| 14/14 [00:03<00:00,  4.19it/s, loss=0.247]


Validation Accuracy: 92.79%

Final Results:
VGG16          : 98.20%
RESNET50       : 98.20%
INCEPTION_V3   : 94.59%
EFFICIENTNET_B0: 92.79%

Best Model: VGG16 with 98.20% accuracy


1. Dataset Overview
The Railway Track Misalignment Detection dataset consists of two image classes:
Normal, Defective

2. Pre-trained Networks Used
We evaluated and fine-tuned the following ImageNet-pretrained models using torchvision.models:

VGG16, ResNet50, InceptionV3, EfficientNet-B0

3. Training Setup
Data Augmentation:
*  Resized to 224x224 (299x299 for InceptionV3)
*  Random horizontal, vertical flips
*  Random rotations (±15-30°)
*  Gaussian Blur applied
*  Color jittering
*  Standard ImageNet normalization

Training Details:

* Optimizer: AdamW
* Scheduler: CosineAnnealingLR
* Loss Function: CrossEntropyLoss with label smoothing = 0.1
* Regularization: MixUp (α = 0.2), Gradient Clipping
* Epochs: 20
* Batch Size: 32
* Device: GPU (CUDA)

4. Architecture Modifications

| Model           | Final Layer Modified                                        |
| --------------- | ----------------------------------------------------------- |
| VGG16           | `classifier[6] → Linear(4096, 2)`                           |
| ResNet50        | `fc → Linear(2048, 2)`                                      |
| InceptionV3     | `fc → Linear(2048, 2)` <br> `AuxLogits.fc → Linear(768, 2)` |
| EfficientNet-B0 | `classifier[1] → Linear(1280, 2)`                           |

5. Performance Comparison

| Model           | Validation Accuracy (%) |
| --------------- | ----------------------- |
| **VGG16**       | **98.20**✅ *(Best)*   |
| ResNet50        | 98.20                   |
| InceptionV3     | 94.59                   |
| EfficientNet-B0 | 92.79                   |

